In [1]:
import pymysql #Connector library for mysql
import re
import nltk
from nltk.tokenize import *
from nltk.corpus import stopwords
from nltk.stem import *
from nltk import *

# Stem the
def stem(word):
    wnl = WordNetLemmatizer()
    temp_text=wnl.lemmatize(word,'v')
    return wnl.lemmatize(temp_text,'n')

# Tokenize
def tokenize(review):
    tok = TreebankWordTokenizer().tokenize(review)
    return tok

# Remove stopwords
def remove_stop(review):
    stopset = set(stopwords.words('english'))
    result_text= [w for w in review if not w in stopset]
    return result_text

#Set up a connection with the server
try:
    conn = pymysql.connect(host = 'localhost', port = 3306, user = 'root', passwd = 'root', db='YELP_DATA_PROJECT')
except:
    print("I am unable to connect to the database")
#And a cursor object that will serve as a virtual 'cursor'
curr = conn.cursor()
extract_usa_review=('Select text from usa_reviews limit 9000;')
curr.execute(extract_usa_review)
results=curr.fetchall()
reviews_text=""
for row in results:
    reviews_text+=row[0]


#Tokenize the reviews -- Takes as input a string
tokenized_reviews=tokenize(reviews_text)

#stem reviews & lower case
stemmed_reviews=""
for word in tokenized_reviews:
    word=word.lower()
    stemmed_reviews+=stem(word) +" "

#Tokenize stemmed text
tok_stem_reviews=tokenize(stemmed_reviews)

#Remove stopwords
tok_stem_stop_reviews=remove_stop(tok_stem_reviews)

#Only keep words with only letters
preprocessed_words=[]
for word in tok_stem_stop_reviews:
    if(word.isalpha()):
        preprocessed_words.append(word)
        
#Part of speech tagging -> Only keep nouns
reviews_nouns=[]
str_nouns=""
pos=nltk.pos_tag(preprocessed_words)
leng = len(pos)
for x in range(0,leng):
    if(pos[x][1]=="NN"):
        reviews_nouns.append(pos[x][0])
        str_nouns+=str(pos[x][0])+" "

        
#Frequency analysis
fdist = FreqDist(reviews_nouns)
top_100=fdist.most_common(100)
print(top_100)


#Wordcloud
import matplotlib.pyplot as plt
!pip install wordcloud
from wordcloud import WordCloud

def word_cloud(text):
    wordcloud = WordCloud().generate(text)
    # Open a plot of the generated image.
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

#Wordcloud of Yelp data
word_cloud(str_nouns)



[('food', 6155), ('place', 5490), ('order', 4245), ('time', 3261), ('service', 3010), ('restaurant', 1765), ('eat', 1724), ('price', 1410), ('get', 1386), ('taste', 1379), ('try', 1355), ('thing', 1265), ('menu', 1179), ('chicken', 1161), ('bar', 1109), ('look', 1078), ('way', 1061), ('burger', 1060), ('fry', 1035), ('night', 1029), ('pizza', 995), ('staff', 990), ('everything', 965), ('lot', 915), ('dinner', 914), ('love', 895), ('star', 889), ('sauce', 888), ('hour', 870), ('meal', 869), ('lunch', 861), ('side', 857), ('something', 829), ('experience', 823), ('day', 791), ('wait', 788), ('meat', 785), ('beer', 763), ('flavor', 755), ('friend', 750), ('sit', 741), ('roll', 729), ('salad', 719), ('drink', 699), ('seat', 674), ('server', 665), ('beef', 657), ('cook', 649), ('visit', 649), ('portion', 645), ('nothing', 643), ('rice', 642), ('review', 622), ('bread', 616), ('location', 603), ('sushi', 597), ('breakfast', 591), ('year', 590), ('steak', 589), ('quality', 586), ('ask', 577),

You are using pip version 7.0.3, however version 7.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
